<a href="https://colab.research.google.com/github/jenmusdat/Project3_BeerQuality/blob/main/Analysis/training_look.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/spark-2.4.7/spark-2.4.7-bin-hadoop2.7.tgz
!tar xf spark-2.4.7-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.7-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:4 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:6 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:7 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:8 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:9 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Ign:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:13 https://developer.download.n

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
# Start a SparkSession
from pyspark.sql import SparkSession
# spark = SparkSession.builder.appName("word2vec").getOrCreate()

In [4]:
MAX_MEMORY = "5g"

spark = SparkSession \
    .builder \
    .appName("word2vec") \
    .config("spark.executor.memory", MAX_MEMORY) \
    .config("spark.driver.memory", MAX_MEMORY) \
    .config("spark.driver.maxResultSize", MAX_MEMORY)\
    .getOrCreate()

In [5]:
# Import needed libs
from pyspark.sql.types import StructType, StructField, StringType
from pyspark.ml.feature import Word2Vec, Word2VecModel
import string


In [6]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url ="/content/gdrive/MyDrive/Project3_BeerQuality/Resources/reviews_beer_brewery.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("reviews_beer_brewery.csv"), header=True)


In [7]:
# remove alpha characters from data set on review numerics
from pyspark.sql.functions import regexp_extract, col
pattern = r'^\d+.\d+$'
df = df.filter(regexp_extract(col('look'), pattern, 0) != '')

In [8]:
df = df.filter(regexp_extract(col('smell'), pattern, 0) != '')
df = df.filter(regexp_extract(col('taste'), pattern, 0) != '')
df = df.filter(regexp_extract(col('overall'), pattern, 0) != '')
df = df.filter(regexp_extract(col('score'), pattern, 0) != '')
df = df.filter(regexp_extract(col('feel'), pattern, 0) != '')

In [9]:
# Next 6 lines casts to floats all the review ratings
df = df.withColumn('look_numeric', df['look'].cast('float'))

In [10]:
df = df.withColumn('smell_numeric', df['smell'].cast('float'))

In [11]:
df = df.withColumn('taste_numeric', df['taste'].cast('float'))

In [12]:
df = df.withColumn('feel_numeric', df['feel'].cast('float'))

In [13]:
df = df.withColumn('overall_numeric', df['overall'].cast('float'))

In [14]:
df = df.withColumn('score_numeric', df['score'].cast('float'))

In [15]:
# Confirm data types
df.printSchema

<bound method DataFrame.printSchema of DataFrame[beer_id: string, username: string, date: string, text: string, look: string, smell: string, taste: string, feel: string, overall: string, score: string, beer_name: string, review_state: string, style: string, availability: string, abv: string, brewery_id: string, brewery_name: string, brewery_city: string, brewery_state: string, brewery_types: string, look_numeric: float, smell_numeric: float, taste_numeric: float, feel_numeric: float, overall_numeric: float, score_numeric: float]>

In [16]:
# functions for manipulating review text into a format we can tokenize
def remove_punctuation(txt):
  txt = txt.strip('\xa0\xa0 ')
  return "".join(l if l not in string.punctuation else "" for l in txt)

def remove_space (text):
  if text:
    return text.replace(' ', '')
  else:
    return 'Unknown'
  

In [17]:
# define the udfs for the data
from pyspark.sql.types import StringType
from pyspark.sql.functions import col, udf

remove_punctuation_udf = udf(remove_punctuation, StringType())
remove_punctuation_udf

remove_space_udf = udf(remove_space, StringType())
remove_space_udf

<function __main__.remove_space>

In [18]:
# clean the beer name by removing spaces (will be combined with text)
beer_df = df.withColumn('beer_clean', remove_space_udf(col('beer_name')))
# beer_df.show()

In [19]:
# clean the style name by removing spaces (will be combined with text)
style_df = beer_df.withColumn('style_clean', remove_space_udf(col('style')))
# style_df.show()

In [20]:
# clean the text by removing punctuation
text_df = style_df.withColumn("clean_text", remove_punctuation_udf(col("text")))
text_df.show()

+-------+----------------+----------+--------------------+----+-----+-----+----+-------+-----+--------------------+------------+--------------------+--------------------+----+----------+--------------------+------------+-------------+-------------+------------+-------------+-------------+------------+---------------+-------------+--------------------+--------------------+--------------------+
|beer_id|        username|      date|                text|look|smell|taste|feel|overall|score|           beer_name|review_state|               style|        availability| abv|brewery_id|        brewery_name|brewery_city|brewery_state|brewery_types|look_numeric|smell_numeric|taste_numeric|feel_numeric|overall_numeric|score_numeric|          beer_clean|         style_clean|          clean_text|
+-------+----------------+----------+--------------------+----+-----+-----+----+-------+-----+--------------------+------------+--------------------+--------------------+----+----------+--------------------+-

In [21]:
# Combine beer_name, style, and review text into a Final_text column
from pyspark.sql.functions import concat_ws

combined_df = text_df.withColumn('Final_text', concat_ws(' ',text_df.beer_clean, text_df.style_clean, text_df.clean_text))
combined_df.select('text', 'Final_text').show(truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [22]:
# Setup and execute Tokenizer and Stop words pipeline
from pyspark.ml.feature import Tokenizer, StopWordsRemover
from pyspark.ml import Pipeline

# "Creating pipeline..."
tokenizer = Tokenizer(inputCol="Final_text", outputCol="token_text")
stopremove = StopWordsRemover(inputCol='token_text', outputCol='features')

pipeline = Pipeline(stages=[tokenizer, stopremove])

# "Training model..."
pipeline_stg = pipeline.fit(combined_df)
final_df = pipeline_stg.transform(combined_df)


In [ ]:
# final_df.show()

In [23]:
final_df = final_df.select('look_numeric','features')

In [24]:
final_df.show()

+------------+--------------------+
|look_numeric|            features|
+------------+--------------------+
|         4.0|[motorbreathimper...|
|         4.0|[paybackpilsner, ...|
|         4.0|[paybackpilsner, ...|
|        3.25|[paybackpilsner, ...|
|        3.75|[paybackpilsner, ...|
|        3.75|[paybackpilsner, ...|
|        3.75|[paybackpilsner, ...|
|        3.75|[paybackpilsner, ...|
|         4.5|[paybackpilsner, ...|
|         3.5|[paybackpilsner, ...|
|         3.5|[hellhammer, ryeb...|
|        4.25|[hellhammer, ryeb...|
|         3.0|[vanillaporter, a...|
|         4.0|[vanillaporter, a...|
|        3.75|[vanillaporter, a...|
|        3.75|[vanillaporter, a...|
|        2.75|[vanillaporter, a...|
|         4.0|[vanillaporter, a...|
|         4.0|[vanillaporter, a...|
|        3.75|[vanillaporter, a...|
+------------+--------------------+
only showing top 20 rows



In [25]:
# Read the model from disk to confirm
w2v_loaded_model = Word2VecModel.load('/content/gdrive/MyDrive/Project3_BeerQuality/review.md')

In [26]:
# check to see if synonym is consistent with loaded model
w2v_loaded_model.findSynonymsArray("light", 5)

[('mild', 0.6426884531974792),
 ('faint', 0.6223998665809631),
 ('slight', 0.6191715002059937),
 ('mostly', 0.6169756650924683),
 ('medium', 0.6095581650733948)]

In [27]:
# Transform the mode
w2v_model_df = w2v_loaded_model.transform(final_df)

In [28]:
w2v_model_df.show()

+------------+--------------------+--------------------+
|look_numeric|            features|               model|
+------------+--------------------+--------------------+
|         4.0|[motorbreathimper...|[0.02950784611367...|
|         4.0|[paybackpilsner, ...|[-0.0872463285923...|
|         4.0|[paybackpilsner, ...|[0.01790586973612...|
|        3.25|[paybackpilsner, ...|[-0.1276244542612...|
|        3.75|[paybackpilsner, ...|[-0.0131208907440...|
|        3.75|[paybackpilsner, ...|[-0.0353247860187...|
|        3.75|[paybackpilsner, ...|[-0.0606238029503...|
|        3.75|[paybackpilsner, ...|[0.00897214795847...|
|         4.5|[paybackpilsner, ...|[0.02342551416502...|
|         3.5|[paybackpilsner, ...|[-0.0209101041158...|
|         3.5|[hellhammer, ryeb...|[0.01432307037573...|
|        4.25|[hellhammer, ryeb...|[-0.1081325657665...|
|         3.0|[vanillaporter, a...|[-0.1101182210924...|
|         4.0|[vanillaporter, a...|[-0.0197092709394...|
|        3.75|[vanillaporter, a

In [29]:
w2v_model_df = w2v_model_df.select('look_numeric', 'model')

In [30]:
# Prepare the hyperparameters
from pyspark.ml.regression import LinearRegression, LinearRegressionModel
lin_model = LinearRegression(
   maxIter=5,
   regParam=0.3,
   featuresCol="model",
   labelCol="look_numeric"
)

In [31]:
w2v_model_df.cache()
w2v_model_df.show()

+------------+--------------------+
|look_numeric|               model|
+------------+--------------------+
|         4.0|[0.02950784611367...|
|         4.0|[-0.0872463285923...|
|         4.0|[0.01790586973612...|
|        3.25|[-0.1276244542612...|
|        3.75|[-0.0131208907440...|
|        3.75|[-0.0353247860187...|
|        3.75|[-0.0606238029503...|
|        3.75|[0.00897214795847...|
|         4.5|[0.02342551416502...|
|         3.5|[-0.0209101041158...|
|         3.5|[0.01432307037573...|
|        4.25|[-0.1081325657665...|
|         3.0|[-0.1101182210924...|
|         4.0|[-0.0197092709394...|
|        3.75|[-0.0049533498369...|
|        3.75|[0.02202844081446...|
|        2.75|[-0.0874559852428...|
|         4.0|[-0.0728758174786...|
|         4.0|[-0.1125914826324...|
|        3.75|[-0.1178625054068...|
+------------+--------------------+
only showing top 20 rows



In [35]:
# split training and testing for the model (for temporary make the training set small as processing takes time)
training, testing = w2v_model_df.randomSplit([0.70, 0.30])

In [36]:
training.show()

+------------+--------------------+
|look_numeric|               model|
+------------+--------------------+
|         1.0|[-0.1452652807929...|
|         1.0|[-0.1452391079316...|
|         1.0|[-0.1097804032266...|
|         1.0|[-0.1004139792794...|
|         1.0|[-0.0699580009898...|
|         1.0|[-0.0677027806639...|
|         1.0|[-0.0668491764137...|
|         1.0|[-0.0606100791133...|
|         1.0|[-0.0502860183739...|
|         1.0|[-0.0460421969907...|
|         1.0|[-0.0457494562491...|
|         1.0|[-0.0451619766827...|
|         1.0|[-0.0447331815958...|
|         1.0|[-0.0392828791445...|
|         1.0|[-0.0315594696129...|
|         1.0|[-0.0313580918803...|
|         1.0|[-0.0309503908132...|
|         1.0|[-0.0279230981555...|
|         1.0|[-0.0214752812858...|
|         1.0|[-0.0148458256818...|
+------------+--------------------+
only showing top 20 rows



In [37]:
training_pandas_df = training.toPandas()

In [38]:
training_pandas_df.isnull().sum()

look_numeric    0
model           0
dtype: int64

In [39]:
training_pandas_df = training_pandas_df.dropna()

In [40]:
training_pandas_df.head()

,look_numeric,model
0,1.0,"[-0.1452652807929553, 0.14236249827081338, 0.0..."
1,1.0,"[-0.14523910793165368, 0.12042828451505759, -0..."
2,1.0,"[-0.10978040322661399, 0.04642222771154982, -0..."
3,1.0,"[-0.10041397927949826, -0.0005390884975592296,..."
4,1.0,"[-0.06995800098982351, 0.17003739844665103, -0..."


In [41]:
training_pandas_df.dtypes

look_numeric    float32
model            object
dtype: object

In [42]:
from sklearn.linear_model import LinearRegression
pandas_lin_model = LinearRegression()

In [44]:
lin_look_train_model = pandas_lin_model.fit(training_pandas_df['model'].to_list(),training_pandas_df['look_numeric'])

In [47]:
lin_look_train_model.score(training_pandas_df['model'].to_list(), training_pandas_df['look_numeric'])

0.28335321235655553

In [48]:
lin_look_train_model.coef_

array([ 0.03612855,  0.95914517,  0.59801143, -0.31203758,  0.40912122,
        0.83162176,  0.38089003,  0.42679738, -0.06415148, -0.16930124,
       -0.05224085, -0.26438838,  1.01482726,  0.06038876,  0.42305088,
        0.61403835, -0.7234493 ,  0.27930889, -0.08511542, -0.35100017,
       -0.04684642, -0.40535001,  0.06756978, -0.04576692, -0.06218717,
       -0.52930928, -0.28897253, -0.5277672 , -0.66238139,  0.81578713,
        0.60313554, -0.27068488,  0.34845423,  0.14721123, -0.41940696,
       -0.47954009, -0.85763678,  0.16164933,  0.15561913, -0.33237642,
        0.23954987, -0.59017379,  0.07929593, -0.83292286, -0.00503841,
       -0.49295991, -0.45474164, -0.65529776, -1.47398722, -0.07185648,
       -0.45585538,  0.44323269,  0.79011083,  0.31210684, -0.26192581,
        0.40658684, -0.42312592, -0.27550912, -0.48307467,  0.11394354,
        0.54606385,  0.41036323, -0.68146095,  0.93229556,  0.45074582,
       -0.20329368,  0.0584169 , -0.60846543, -0.48700248, -0.36

In [50]:
# Fit the score review to a model (takes time)
lin_look_train_model = lin_model.fit(training)

In [51]:
# Save the model for later use
lin_look_train_model.write().overwrite().save("/content/gdrive/MyDrive/Project3_BeerQuality/lin_look_train.md")

In [52]:
# Read the model
loaded_lin_look_train_model = LinearRegressionModel.load("/content/gdrive/MyDrive/Project3_BeerQuality/lin_look_train.md")

In [53]:
# Transform the training data to peek at the predictions
lin_look_df = loaded_lin_look_train_model.transform(training)

In [54]:
lin_look_df.show()

+------------+--------------------+------------------+
|look_numeric|               model|        prediction|
+------------+--------------------+------------------+
|         1.0|[-0.1452652807929...|  3.30244608916442|
|         1.0|[-0.1452391079316...| 3.536130651421115|
|         1.0|[-0.1097804032266...|3.2490701248863076|
|         1.0|[-0.1004139792794...| 3.166858788165892|
|         1.0|[-0.0699580009898...|3.6137354258189363|
|         1.0|[-0.0677027806639...| 4.272408374875637|
|         1.0|[-0.0668491764137...|4.0272505656809345|
|         1.0|[-0.0606100791133...| 3.238519677525181|
|         1.0|[-0.0502860183739...| 3.429430468509798|
|         1.0|[-0.0460421969907...| 4.000470709707322|
|         1.0|[-0.0457494562491...|3.7905424398357046|
|         1.0|[-0.0451619766827...|3.5144856162288294|
|         1.0|[-0.0447331815958...|3.9188899676838815|
|         1.0|[-0.0392828791445...| 3.131606853889373|
|         1.0|[-0.0315594696129...| 4.350214981691045|
|         

In [55]:
# show a sample of the predictions
lin_look_df.select('look_numeric', 'prediction').show(truncate=False)

+------------+------------------+
|look_numeric|prediction        |
+------------+------------------+
|1.0         |3.30244608916442  |
|1.0         |3.536130651421115 |
|1.0         |3.2490701248863076|
|1.0         |3.166858788165892 |
|1.0         |3.6137354258189363|
|1.0         |4.272408374875637 |
|1.0         |4.0272505656809345|
|1.0         |3.238519677525181 |
|1.0         |3.429430468509798 |
|1.0         |4.000470709707322 |
|1.0         |3.7905424398357046|
|1.0         |3.5144856162288294|
|1.0         |3.9188899676838815|
|1.0         |3.131606853889373 |
|1.0         |4.350214981691045 |
|1.0         |3.608981177587155 |
|1.0         |3.5418561012745715|
|1.0         |3.7564755367666494|
|1.0         |3.2580726093425487|
|1.0         |3.4855895797002328|
+------------+------------------+
only showing top 20 rows



In [56]:
gr = lin_look_df.groupBy("look_numeric").count()
gr.show(100)

+------------+------+
|look_numeric| count|
+------------+------+
|         5.0| 57836|
|         2.5| 19100|
|         2.0| 12020|
|         3.0| 92637|
|        2.75|  4415|
|         1.5|  3021|
|        3.75| 86875|
|        1.75|   577|
|        4.25|105556|
|         3.5|211384|
|         1.0|  2457|
|        2.25|  1353|
|        4.75| 30774|
|         4.5|220111|
|        1.25|   344|
|        3.25| 19042|
|         4.0|512540|
+------------+------+



In [57]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('To enable a high-RAM runtime, select the Runtime > "Change runtime type"')
  print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
  print('re-execute this cell.')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.4 gigabytes of available RAM

You are using a high-RAM runtime!
